In [1]:
!pip install uproot

In [2]:
!pip install -q torch-scatter
!pip install -q torch-sparse
!pip install -q torch-geometric

In [3]:
!pip install uproot tqdm numexpr google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.7/380.7 kB 5.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Preparing metadata (setup.py) ... done
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached portpicker-1.2.0.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached tornado-4.5.3.tar.gz (484 kB)
  Preparing metadata (setup.py) ... done
  Using cached simplegeneric-0.8.1.zip (12 kB)
  Preparing metadata (setup.py) ... done
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.wh

  Created wheel for google-colab: filename=google_colab-1.0.0-py2.py3-none-any.whl size=102275 sha256=691e36bc6eb5959759821a8b4d9687797148c4fcfcddf322421d7147b1180bef
  Stored in directory: /home/jonas/.cache/pip/wheels/cf/81/3b/a50a90282756a5257b8885aeb7d22de09c3bfbf31d66ff455e
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1079 lines of output]
      /tmp/pip-install-fj0hoaeu/pandas_b496d5f484c7452892f11d15641c1a44/setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
        _CYTHON_INSTALLED = ver >= LooseVersion(min_cython_ver)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/pandas
      copying pandas/__init__.py -> build/lib.linux-x86_64-cpython-39/pandas
      copying pandas/testing.py -> build/lib.linux-x86_64-cpy

  Created wheel for portpicker: filename=portpicker-1.2.0-py3-none-any.whl size=13384 sha256=0261a463cf8b3d25ae93e74553d2cc031bbe32e25eb439dbdb39412c621de5ca
  Stored in directory: /home/jonas/.cache/pip/wheels/d3/dc/cc/bace7ffe874129f722f8eee56e162461907e1976cf1102d84c
  Created wheel for tornado: filename=tornado-4.5.3-cp39-cp39-linux_x86_64.whl size=423430 sha256=6300f1abc1b1dc81027d8e8f4110692e35d1b1cbfef199b42cb8269ea6787a47
  Stored in directory: /home/jonas/.cache/pip/wheels/ae/3b/7c/05b9d47da5eba9d22cbba0207676ef11072f119db2ff797352
  Created wheel for simplegeneric: filename=simplegeneric-0.8.1-py3-none-any.whl size=5059 sha256=159ca4ba287bd776d1135b5a0097a1b299ae172c17326c4e464a267cce3b4467
  Stored in directory: /home/jonas/.cache/pip/wheels/57/40/23/f22872052dd8633ca3672b2cc424fd116564dcee8a187710a1
Successfully built google-colab portpicker tornado simplegeneric
Failed to build pandas
  Attempting uninstall: tornado
    Found existing installation: tornado 6.3.1
    Uninst

      gcc -pthread -B /home/jonas/mambaforge/envs/dfei39/compiler_compat -shared -Wl,-rpath,/home/jonas/mambaforge/envs/dfei39/lib -Wl,-rpath-link,/home/jonas/mambaforge/envs/dfei39/lib -L/home/jonas/mambaforge/envs/dfei39/lib -L/home/jonas/mambaforge/envs/dfei39/lib -Wl,-rpath,/home/jonas/mambaforge/envs/dfei39/lib -Wl,-rpath-link,/home/jonas/mambaforge/envs/dfei39/lib -L/home/jonas/mambaforge/envs/dfei39/lib build/temp.linux-x86_64-cpython-39/pandas/_libs/index.o build/temp.linux-x86_64-cpython-39/pandas/_libs/tslibs/src/datetime/np_datetime.o build/temp.linux-x86_64-cpython-39/pandas/_libs/tslibs/src/datetime/np_datetime_strings.o -o build/lib.linux-x86_64-cpython-39/pandas/_libs/index.cpython-39-x86_64-linux-gnu.so
      building 'pandas._libs.indexing' extension
      gcc -pthread -B /home/jonas/mambaforge/envs/dfei39/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/jonas/mambaforge/envs/dfei39/include -I/home/jonas/mambaforge/envs/dfei

In [4]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras import initializers
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import pickle
import torch
from torch_geometric.data import Data
import torch_geometric
import tqdm
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.loader import DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [5]:
particles_file = uproot.open("/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/particles_data.root")

In [6]:
particles_tree = particles_file['Particles']

In [7]:
relevant_particles_keys = ['ExpandedEventNumber',
                          'ExpandedParticleIndex',
                          'ExpandedPrimaryHeavyHadronIndex',
                          'FromPrimaryHeavyHadron',
                          'ParticleType',
                          'associated_ip_reco',
                          'associated_xPV_reco',
                          'associated_yPV_reco',
                          'associated_zPV_reco',
                          'charge',
                          'inGeomAcc',
                          'isCharged',
                          'pt_reco',
                          'px_reco',
                          'py_reco',
                          'pz_reco',
                          'xProd_reco',
                          'yProd_reco',
                          'zProd_reco']

In [8]:
df_particles = particles_tree.arrays(relevant_particles_keys, library='pd')

In [9]:
df_particles

,ExpandedEventNumber,ExpandedParticleIndex,ExpandedPrimaryHeavyHadronIndex,FromPrimaryHeavyHadron,ParticleType,associated_ip_reco,associated_xPV_reco,associated_yPV_reco,associated_zPV_reco,charge,inGeomAcc,isCharged,pt_reco,px_reco,py_reco,pz_reco,xProd_reco,yProd_reco,zProd_reco
0,0.0,1057.0,-1.0,0.0,2.0,-1.000000,-1.000000,-1.000000,-1.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1106.0,-1.0,0.0,0.0,0.015194,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.286947,-0.153205,-0.242625,2.750225,-0.642510,-0.880182,-19.230768
2,0.0,1110.0,-1.0,0.0,0.0,0.006571,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.576406,-0.555643,-0.153313,13.076964,-0.511701,-0.120768,-19.230768
3,0.0,1111.0,-1.0,0.0,0.0,0.010698,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.922732,-0.865247,0.320597,6.068925,-1.524306,0.530222,-19.230768
4,0.0,1112.0,-1.0,0.0,0.0,0.010318,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.253314,-0.246892,0.056676,1.878958,-1.392859,0.292333,-19.230768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374328,6161.0,2430.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.657934,-0.611031,0.243963,3.661947,0.000000,0.000000,0.000000
1374329,6161.0,2431.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.559823,-0.523263,-0.198992,2.484762,0.000000,0.000000,0.000000
1374330,6161.0,2444.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.313306,-0.313267,0.004922,1.603106,0.000000,0.000000,0.000000
1374331,6161.0,2445.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.416930,-0.082793,0.408627,2.115970,0.000000,0.000000,0.000000


In [10]:
df_Y = df_particles.loc[:, ['ExpandedEventNumber', 'ExpandedParticleIndex', 'FromPrimaryHeavyHadron']]

In [11]:
df_Y

,ExpandedEventNumber,ExpandedParticleIndex,FromPrimaryHeavyHadron
0,0.0,1057.0,0.0
1,0.0,1106.0,0.0
2,0.0,1110.0,0.0
3,0.0,1111.0,0.0
4,0.0,1112.0,0.0
...,...,...,...
1374328,6161.0,2430.0,0.0
1374329,6161.0,2431.0,0.0
1374330,6161.0,2444.0,0.0
1374331,6161.0,2445.0,0.0


In [12]:
df_particles = df_particles.set_index('ExpandedParticleIndex')

In [13]:
df_particles

,ExpandedEventNumber,ExpandedPrimaryHeavyHadronIndex,FromPrimaryHeavyHadron,ParticleType,associated_ip_reco,associated_xPV_reco,associated_yPV_reco,associated_zPV_reco,charge,inGeomAcc,isCharged,pt_reco,px_reco,py_reco,pz_reco,xProd_reco,yProd_reco,zProd_reco
ExpandedParticleIndex,,,,,,,,,,,,,,,,,,
1057.0,0.0,-1.0,0.0,2.0,-1.000000,-1.000000,-1.000000,-1.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1106.0,0.0,-1.0,0.0,0.0,0.015194,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.286947,-0.153205,-0.242625,2.750225,-0.642510,-0.880182,-19.230768
1110.0,0.0,-1.0,0.0,0.0,0.006571,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.576406,-0.555643,-0.153313,13.076964,-0.511701,-0.120768,-19.230768
1111.0,0.0,-1.0,0.0,0.0,0.010698,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.922732,-0.865247,0.320597,6.068925,-1.524306,0.530222,-19.230768
1112.0,0.0,-1.0,0.0,0.0,0.010318,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.253314,-0.246892,0.056676,1.878958,-1.392859,0.292333,-19.230768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430.0,6161.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.657934,-0.611031,0.243963,3.661947,0.000000,0.000000,0.000000
2431.0,6161.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.559823,-0.523263,-0.198992,2.484762,0.000000,0.000000,0.000000
2444.0,6161.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.313306,-0.313267,0.004922,1.603106,0.000000,0.000000,0.000000


In [14]:
num_of_events = 6161

In [15]:
relevant_edge_features = [
                          'DOCA_reco',
                          'FromSameAssociatedPV_reco',
                          'delta_z0_reco',
                          'p1_isCharged',
                          'p2_isCharged',
                          'theta_reco',
                          'trdist_reco'
                          ]

In [16]:
K = 5

In [17]:
list_of_graphs = []

In [20]:
for event_number in tqdm.tqdm(range(num_of_events)):
  
  df_p = df_particles.loc[df_particles.ExpandedEventNumber == event_number]
  mapping = {index: i for i, index in enumerate(df_p.index)}
  x = df_p.loc[:, ~df_p.columns.isin(['ExpandedParticleIndex', 'ExpandedEventNumber', 'FromPrimaryHeavyHadron', 'ExpandedPrimaryHeavyHadronIndex', 'ParticleType', 'inGeomAcc'])]
  x = torch.tensor(x.values, dtype=torch.float)

  filename = f"event_{event_number}_top_{K}_relations_filtered_by_DOCA_for_each_particle.csv"
  path = f'/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/filtered_relations_by_DOCA/{filename}'
  df_r = pd.read_csv(path)
  edges = []
  edge_attr = []
  for _, row in df_r.iterrows():
    sender = mapping[row['FirstParticleIndex']]
    reciever = mapping[row['SecondParticleIndex']]
    edge_1 = [sender, reciever]
    edge_2 = [reciever, sender]
    edges.append(edge_1)
    edges.append(edge_2)
    edge_features = list(row[relevant_edge_features].values)
    edge_attr.append(edge_features)
    edge_attr.append(edge_features)
  edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
  edge_attr = torch.tensor(edge_attr, dtype=torch.float)


  y = torch.tensor(df_Y.loc[df_Y.ExpandedEventNumber == event_number]['FromPrimaryHeavyHadron'].values)
  y = y.reshape(-1, 1)

  graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

  list_of_graphs.append(graph)

100%|███████████████████████████████████████████████████████████████████████████████| 6161/6161 [40:07<00:00,  2.56it/s]


In [21]:
import pickle 

with open('/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/list_of_graphs_from_filtered_data_k5.pickle', 'wb') as handle:
    pickle.dump(list_of_graphs, handle, protocol=pickle.HIGHEST_PROTOCOL)